In [11]:
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler
import torch
from torch.nn import MultiheadAttention
from torch import optim
import torch.nn as nn

device = "cuda:0"

model = TabPFNClassifier(device=device)
module = model.model[2]

Using style prior: True
{'recompute_attn': True}
<module 'wandb' from '/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn2/lib/python3.9/site-packages/wandb/__init__.py'>
Using cuda:0 device
Batch size: 1
Using distributed training: False
Using a Transformer with 25.82 M parameters


In [12]:
from tabpfn.utils import normalize_data, to_ranking_low_mem, remove_outliers
from tabpfn.priors.utils import normalize_by_used_features_f

normalize_with_test = True #TODO change
#eval_position = 100
normalize_to_ranking = False
max_features=100
normalize_with_sqrt = False

def preprocess_input(eval_xs, eval_ys, categorical_feats, preprocess_transform):
    import warnings

    if eval_xs.shape[1] > 1:
        raise Exception("Transforms only allow one batch dim - TODO")
    if preprocess_transform != 'none':
        if preprocess_transform == 'power' or preprocess_transform == 'power_all':
            pt = PowerTransformer(standardize=True)
        elif preprocess_transform == 'quantile' or preprocess_transform == 'quantile_all':
            pt = QuantileTransformer(output_distribution='normal')
        elif preprocess_transform == 'robust' or preprocess_transform == 'robust_all':
            pt = RobustScaler(unit_variance=True)

    # eval_xs, eval_ys = normalize_data(eval_xs), normalize_data(eval_ys)
    eval_xs = normalize_data(eval_xs, normalize_positions=-1 if normalize_with_test else eval_position)

    # Removing empty features
    eval_xs = eval_xs[:, 0, :]
    sel = [len(torch.unique(eval_xs[0:eval_ys.shape[0], col])) > 1 for col in range(eval_xs.shape[1])]
    eval_xs = eval_xs[:, sel]

    warnings.simplefilter('error')
    if preprocess_transform != 'none':
        eval_xs = eval_xs.cpu().numpy()
        feats = set(range(eval_xs.shape[1])) if 'all' in preprocess_transform else set(
            range(eval_xs.shape[1])) - set(categorical_feats)
        for col in feats:
            try:
                pt.fit(eval_xs[0:eval_position, col:col + 1])
                trans = pt.transform(eval_xs[:, col:col + 1])
                # print(scipy.stats.spearmanr(trans[~np.isnan(eval_xs[:, col:col+1])], eval_xs[:, col:col+1][~np.isnan(eval_xs[:, col:col+1])]))
                eval_xs[:, col:col + 1] = trans
            except:
                pass
        eval_xs = torch.tensor(eval_xs).float()
    warnings.simplefilter('default')

    eval_xs = eval_xs.unsqueeze(1)

    # TODO: Cautian there is information leakage when to_ranking is used, we should not use it
    eval_xs = remove_outliers(eval_xs, normalize_positions=-1 if normalize_with_test else eval_position) if not normalize_to_ranking else normalize_data(to_ranking_low_mem(eval_xs))
    # Rescale X
    eval_xs = normalize_by_used_features_f(eval_xs, eval_xs.shape[-1], max_features,
                                            normalize_with_sqrt=normalize_with_sqrt)

    return eval_xs.detach().to(device)


In [13]:
def compute_accuracy_rf(X, y, n_train):
    # Move X and y to CPU
    X = X.to("cpu")
    y = y.to("cpu")
    rf = RandomForestClassifier()
    X_train, X_test = X[:n_train], X[n_train:]
    y_train, y_test = y[:n_train], y[n_train:]
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    return accuracy_score(y_test, y_pred)

def compute_accuracy_tabpfn_original(X, y, n_train):
    X = X.to("cpu") #FIXME
    y = y.to("cpu")
    X_train, X_test = X[:n_train], X[n_train:]
    y_train, y_test = y[:n_train], y[n_train:]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

def compute_accuracy_tabpfn(X, y, n_train, preprocess=False):
    # reshape X to be 3D
    X = X.reshape(X.shape[0], 1, X.shape[1])
    # reshape y to be 2D
    y = y.reshape(y.shape[0], 1)
    if preprocess:
        X = preprocess_input(X, y, [], 'power')
    X = X.to(device)
    y_pred = torch.argmax(module((X, y), single_eval_pos=n_train, x_already_encoded=False), dim=-1).reshape(-1)
    # Check the accuracy
    print(accuracy_score(y[-len(y_pred):].cpu(), y_pred.cpu())) #FIXME
    

In [19]:
def train_attention(module, X1, X2, y, n_train, preprocess=False, lr=0.01, n_epochs=100, train_tabpfn=False):
    # reshape X to be 3D
    X1 = X1.reshape(X1.shape[0], 1, X1.shape[1])
    X2 = X2.reshape(X2.shape[0], 1, X2.shape[1])
    # reshape y to be 2D
    y = y.reshape(y.shape[0], 1)
    if preprocess:
        X1 = preprocess_input(X1, y, [], 'power')
        X2 = preprocess_input(X2, y, [], 'power')

    n = 512 # to be compatible with pretrained TabPFN
    attention = MultiheadAttention(embed_dim=n, num_heads=1, dropout=0.0, device=device)
    W_q = nn.Linear(n, n).to(device)
    W_k = nn.Linear(n, n).to(device)
    W_v = nn.Linear(n, n).to(device)
    #W_o = nn.Linear(n, 2).to(device)
    W_q.requires_grad = True
    W_k.requires_grad = True
    W_v.requires_grad = True
    #W_o.requires_grad = True
    train_size = n_train
    #TODO think about this. Do I really need a train set?
    val_size = (len(X1) - train_size) // 2
    test_size = len(X1) - train_size - val_size
    if train_tabpfn:
        optimizer = optim.Adam([*W_q.parameters(), *W_k.parameters(), *W_v.parameters(), *module.parameters()], lr=lr)
    else:
        optimizer = optim.Adam([*W_q.parameters(), *W_k.parameters(), *W_v.parameters()], lr=lr)
        with torch.no_grad():
            input = module.encoder(X1)
            input2 = module.encoder(X2)
    for i in range(n_epochs):
        optimizer.zero_grad()
        if train_tabpfn:
            input = module.encoder(X1)
            input2 = module.encoder(X2)
        Q = W_q(input)
        K = W_k(input2)
        V = W_v(input2)
        #full_input = input + attention(Q, K, V)[0]
        full_input = input + attention(Q, K, V)[0]
        y_pred = module((full_input, y), single_eval_pos=train_size, x_already_encoded=True).squeeze()
        #y_pred = W_o(full_input).squeeze()
        loss = torch.nn.functional.cross_entropy(y_pred[:val_size], y[-len(y_pred):-len(y_pred) + val_size].squeeze().long())
        loss.backward()
        optimizer.step()
        if i % 2 == 0:
            print(loss)
        if i % 10 == 0:
            y_pred = torch.argmax(y_pred, dim=-1).reshape(-1)
            print("Test accuracy:")
            print("Number of test samples: ", test_size)
            print(accuracy_score(y[-len(y_pred) + val_size:].cpu(), y_pred[val_size:].cpu()))
            print("Train accuracy:")
            #print("Number of train samples: ", train_size)
            #print(accuracy_score(y[:train_size].cpu(), y_pred[:train_size].cpu()))
            print("Val accuracy:")
            print("Number of val samples: ", val_size)
            print(accuracy_score(y[-len(y_pred):-len(y_pred) + val_size].cpu(), y_pred[:val_size].cpu()))

    
    

# Toy example

In [15]:
# create a non linear function
#f = torch.functional.F.relu
f = lambda x: x**2

In [16]:
# Create fake datasets
# Create a first dataset with columns A, B
torch.manual_seed(40)
dataset1 = torch.rand(20000, 100)
# Create a second dataset with columns C, D
dataset2 = torch.rand(20000, 100)

# Make column C equal to column A * 2 + noise
dataset2[:, 0] = dataset1[:, 0]# * 2 #+ torch.rand(1000) * 0.1

# Create y as D + noise
#y = torch.sum(dataset2, dim=1)# + f(dataset1[:, 4]) #+ torch.rand(1000) * 0.1
#with torch.no_grad():
#    y = torch.sum(module.encoder(dataset2.reshape(dataset2.shape[0], 1, dataset2.shape[1]).to(device)).reshape(dataset2.shape[0], -1), dim=1).cpu()
y = dataset2[:, 1]

#dataset2.requires_grad = True
#dataset1.requires_grad = True

# Make it a classification problem
y = (y > np.median(y)).float()

print(np.unique(y, return_counts=True))

# Move to GPU
dataset1 = dataset1.to(device)
dataset2 = dataset2.to(device)
y = y.to(device)
# Shuffle dataset2
dataset2 = dataset2[torch.randperm(len(dataset2))]

(array([0., 1.], dtype=float32), array([10000, 10000]))


In [17]:
print("-- Random Forest --")
print("Accuracy with dataset1", compute_accuracy_rf(dataset1, y, 1000))
print("Accuracy with dataset2", compute_accuracy_rf(dataset2, y, 1000))
print("Accuracy with dataset1 + dataset2", compute_accuracy_rf(torch.cat((dataset1, dataset2), dim=1), y, 1000))

-- Random Forest --
Accuracy with dataset1 0.5012631578947369
Accuracy with dataset2 0.4998947368421053
Accuracy with dataset1 + dataset2 0.4986315789473684


In [7]:
# match indices of dataset1 and dataset2 on the first coordinate
distance = torch.cdist(dataset1[:, 0].reshape(-1, 1), dataset2[:, 0].reshape(-1, 1))
# replace the 2nd coordinate of dataset1 by the 2nd coordinate of dataset2
dataset1_copy = dataset1.clone()
dataset1_copy[:, 1] = torch.sum(dataset2[distance.argmin(dim=1)], dim=1)


print("-- Random Forest --")
print("Accuracy with dataset1", compute_accuracy_rf(dataset1_copy, y, 1000))
print("Accuracy with dataset2", compute_accuracy_rf(dataset2, y, 1000))


-- Random Forest --
Accuracy with dataset1 0.4996842105263158
Accuracy with dataset2 0.49857894736842107


In [8]:
print("-- TabPFN original --")
print("Accuracy with dataset1", compute_accuracy_tabpfn_original(dataset1, y, 1000))
print("Accuracy with dataset2", compute_accuracy_tabpfn_original(dataset2, y, 1000))

-- TabPFN original --
interface
torch.Size([20000, 1, 100])
torch.Size([20000, 1])
1000
Accuracy with dataset1 0.4996315789473684
interface
torch.Size([20000, 1, 100])
torch.Size([20000, 1])
1000
Accuracy with dataset2 0.5006315789473684


In [9]:
print("-- TabPFN --")
print("Without preprocessing")
print("Accuracy with dataset1", compute_accuracy_tabpfn(dataset1, y, 1000, preprocess=False))
print("Accuracy with dataset2", compute_accuracy_tabpfn(dataset2, y, 1000, preprocess=False))
print("With preprocessing")
print("Accuracy with dataset1", compute_accuracy_tabpfn(dataset1, y, 1000, preprocess=True))
print("Accuracy with dataset2", compute_accuracy_tabpfn(dataset2, y, 1000, preprocess=True))

-- TabPFN --
Without preprocessing
0.496
Accuracy with dataset1 None
0.5008947368421053
Accuracy with dataset2 None
With preprocessing
0.4991052631578947
Accuracy with dataset1 None
0.4993684210526316
Accuracy with dataset2 None


In [21]:
train_attention(module, dataset1, dataset2, y, 1000, preprocess=True, lr=0.001, n_epochs=1000, train_tabpfn=False)

tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
Test accuracy:
Number of test samples:  9500
0.4985263157894737
Train accuracy:
Val accuracy:
Number of val samples:  9500
0.5025263157894737
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
Test accuracy:
Number of test samples:  9500
0.4985263157894737
Train accuracy:
Val accuracy:
Number of val samples:  9500
0.5025263157894737
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward>)
Test accuracy:
Number of test samples:  9500
0.4985263157894737
Tra

# Real data

In [248]:
device = "cuda:0"
model = TabPFNClassifier(device=device)

module = model.model[2]

Using style prior: True
{'recompute_attn': True}
<module 'wandb' from '/home/parietal/lgrinszt/.local/miniconda3/envs/tab_pfn2/lib/python3.9/site-packages/wandb/__init__.py'>
Using cuda:0 device
Batch size: 1
Using distributed training: False
Using a Transformer with 25.82 M parameters


## California

In [228]:
# Try the same thing on the california housing dataset
from sklearn.datasets import fetch_california_housing
import pandas as pd

data = fetch_california_housing(as_frame=True)

In [229]:
df_california = data["frame"]
# drop the target
df_california = df_california.drop("MedHouseVal", axis=1)
# drop income
df_california_without_income = df_california.drop("MedInc", axis=1)
# Drop more columns to make it harder
# df_california = df_california.drop("AveOccup", axis=1)
# df_california_without_income = df_california_without_income.drop("AveOccup", axis=1)
# only keep Latitude and Longitude
df_california = df_california[["MedInc", "Latitude", "Longitude"]]
df_california_without_income = df_california_without_income[["Latitude", "Longitude"]]
target = data["target"]

In [230]:
df_california

,MedInc,Latitude,Longitude
0,8.3252,37.88,-122.23
1,8.3014,37.86,-122.22
2,7.2574,37.85,-122.24
3,5.6431,37.85,-122.25
4,3.8462,37.85,-122.25
...,...,...,...
20635,1.5603,39.48,-121.09
20636,2.5568,39.49,-121.21
20637,1.7000,39.43,-121.22
20638,1.8672,39.43,-121.32


In [231]:
data = {'City': ['Los Angeles', 'San Diego', 'San Jose', 'San Francisco', 'Fresno', 'Sacramento', 'Long Beach', 'Oakland', 'Bakersfield', 'Anaheim', 'Santa Ana', 'Riverside', 'Stockton', 'Chula Vista', 'Irvine', 'Fremont', 'San Bernardino', 'Modesto', 'Fontana', 'Oxnard'], 
        'Average Household Income': ['$101,006', '$113,681', '$150,601', '$167,663', '$73,396', '$87,213', '$89,912', '$116,585', '$84,592', '$97,136', '$88,829', '$90,520', '$78,712', '$105,155', '$140,764', '$170,083', '$64,929', '$81,841', '$93,383', '$91,636']}

df_cities = pd.DataFrame(data)

In [232]:
# Get lat and long for each city
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application (leo.grinsztajn@gmail.com)")
df_cities["location"] = df_cities["City"].apply(lambda x: geolocator.geocode(x))
df_cities["latitude"] = df_cities["location"].apply(lambda x: x.latitude)
df_cities["longitude"] = df_cities["location"].apply(lambda x: x.longitude)


### Baselines

In [1]:
# Convert target to a classification problem
target_classif = (target > np.median(target)).astype(int)

# Convert df_california to a torch tensor
california_tensor = torch.tensor(df_california.values).float()
california_without_income_tensor = torch.tensor(df_california_without_income.values).float()
target_tensor = torch.tensor(target_classif.values).float()

# Suffle the data
from sklearn.utils import shuffle
california_tensor, california_without_income_tensor, target_tensor = shuffle(california_tensor, california_without_income_tensor, target_tensor, random_state=0)

NameError: name 'target' is not defined

In [ ]:
# Reshape
california_tensor = california_tensor.reshape(california_tensor.shape[0], 1, california_tensor.shape[1])
california_without_income_tensor = california_without_income_tensor.reshape(california_without_income_tensor.shape[0], 1, california_without_income_tensor.shape[1])
target_tensor = target_tensor.reshape(target.shape[0], 1)

In [235]:
# Truncate the first dim to 5000
california_tensor = california_tensor[:5000]
california_without_income_tensor = california_without_income_tensor[:5000]
target_tensor = target_tensor[:5000]

print(np.unique(target_tensor, return_counts=True))
# print proportion of 1
p = np.unique(target_tensor, return_counts=True)[1][1] / np.unique(target_tensor, return_counts=True)[1].sum()
print(p)
print(1-p)


(array([0., 1.], dtype=float32), array([2549, 2451]))
0.4902
0.5098


In [236]:
print(california_tensor.shape)
print(california_without_income_tensor.shape)

torch.Size([5000, 1, 3])
torch.Size([5000, 1, 2])


In [237]:
# Move to GPU
california_tensor = california_tensor.to(device)
california_without_income_tensor = california_without_income_tensor.to(device)
target_tensor = target_tensor.to(device)

In [252]:
#model = TabPFNClassifier(N_ensemble_configurations=1, feature_shift_decoder=False)

#module = model.model[2]
# Pad with 0s until we have 100 features

n_samples, n_features = california_tensor.shape[0], california_tensor.shape[2]
model.fit(california_tensor[:1000].reshape(1000, n_features).cpu(), target_tensor[:1000].reshape(1000).cpu())
y_pred = model.predict(california_tensor[1000:].reshape(4000, n_features).cpu())
print("Accuracy with income")
print(accuracy_score(target_tensor[1000:].reshape(-1).cpu(), y_pred))

n_samples, n_features = california_without_income_tensor.shape[0], california_without_income_tensor.shape[2]
model.fit(california_without_income_tensor[:1000].reshape(1000, n_features).cpu(), target_tensor[:1000].reshape(1000).cpu())
y_pred = model.predict(california_without_income_tensor[1000:].reshape(4000, n_features).cpu())
print("Accuracy without income")
print(accuracy_score(target_tensor[1000:].reshape(-1).cpu(), y_pred))





interface
torch.Size([5000, 1, 3])
torch.Size([5000, 1])
1000
src_mask None
Accuracy with income
0.8215
interface
torch.Size([5000, 1, 2])
torch.Size([5000, 1])
1000
src_mask None
Accuracy without income
0.77825


In [256]:
# Preprocess the data
from sklearn.preprocessing import StandardScaler, PowerTransformer, QuantileTransformer
#TODO why the difference with random forest?
#scaler = StandardScaler()
#california_tensor_processed = torch.tensor(scaler.fit_transform(california_tensor.reshape(-1, california_tensor.shape[-1])).reshape(california_tensor.shape))
#california_without_income_tensor_processed = torch.tensor(scaler.fit_transform(california_without_income_tensor.reshape(-1, california_without_income_tensor.shape[-1])).reshape(california_without_income_tensor.shape))
print(california_tensor.shape)
california_tensor_processed = preprocess_input(california_tensor, target_tensor, [], preprocess_transform='power')
california_without_income_tensor_processed = preprocess_input(california_without_income_tensor, target_tensor, [], preprocess_transform='power')
# Pad last dimension with 0 to 100
california_tensor_processed = torch.nn.functional.pad(california_tensor_processed, (0, 100-california_tensor.shape[2]))
california_without_income_tensor_processed = torch.nn.functional.pad(california_without_income_tensor_processed, (0, 100-california_without_income_tensor.shape[2]))
print(california_tensor_processed.shape)
# convert to float
california_tensor_processed = california_tensor_processed.float()
y_pred = torch.argmax(module((california_tensor_processed, target_tensor), single_eval_pos=1000, x_already_encoded=False), dim=-1).reshape(-1)
california_without_income_tensor_processed = california_without_income_tensor_processed.float()
y_pred2 = torch.argmax(module((california_without_income_tensor_processed, target_tensor), single_eval_pos=1000, x_already_encoded=False), dim=-1).reshape(-1)
# print shapes
print(y_pred.shape)
print(y_pred2.shape)

from sklearn.metrics import accuracy_score
print("Accuracy with income")
print(accuracy_score(target_tensor[-len(y_pred):].cpu(), y_pred.cpu()))
print("Accuracy without income")
print(accuracy_score(target_tensor[-len(y_pred2):].cpu(), y_pred2.cpu()))

torch.Size([5000, 1, 3])
torch.Size([5000, 1, 100])
src_mask None
src_mask None
torch.Size([4000])
torch.Size([4000])
Accuracy with income
0.822
Accuracy without income
0.783


In [258]:
# Try with a random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
X, y = california_tensor.reshape(california_tensor.shape[0], -1).cpu().numpy(), target_tensor.reshape(-1).cpu().numpy()
# The train part is the first 1000 samples
rf.fit(X[:1000], y[:1000])
y_pred = rf.predict(X[1000:])
print("Accuracy with income")
print(accuracy_score(y[1000:], y_pred))

# without income
rf = RandomForestClassifier()
X, y = california_without_income_tensor.reshape(california_without_income_tensor.shape[0], -1).cpu().numpy(), target_tensor.reshape(-1).cpu().numpy()
# The train part is the first 1000 samples
rf.fit(X[:1000], y[:1000])
y_pred = rf.predict(X[1000:])
print("Accuracy without income")
print(accuracy_score(y[1000:], y_pred))



Accuracy with income
0.84425
Accuracy without income
0.835


### Supervised join

In [259]:
df_cities

,City,Average Household Income,location,latitude,longitude
0,Los Angeles,"$101,006","(Los Angeles, Los Angeles County, CAL Fire Con...",34.053691,-118.242766
1,San Diego,"$113,681","(San Diego, San Diego County, CAL Fire Souther...",32.717420,-117.162773
2,San Jose,"$150,601","(San Jose, Santa Clara County, CAL Fire Northe...",37.336166,-121.890591
3,San Francisco,"$167,663","(San Francisco, CAL Fire Northern Region, Cali...",37.779026,-122.419906
4,Fresno,"$73,396","(Fresno, Fresno County, CAL Fire Southern Regi...",36.739442,-119.784831
5,Sacramento,"$87,213","(Sacramento, Sacramento County, CAL Fire North...",38.581061,-121.493895
6,Long Beach,"$89,912","(Long Beach, Los Angeles County, CAL Fire Cont...",33.769016,-118.191604
7,Oakland,"$116,585","(Oakland, Alameda County, CAL Fire Northern Re...",37.804456,-122.271356
8,Bakersfield,"$84,592","(Bakersfield, Kern County, CAL Fire Southern R...",35.373871,-119.019464
9,Anaheim,"$97,136","(Anaheim, Orange County, CAL Fire Contract Cou...",33.834752,-117.911732


In [260]:
#convert value like $101,006' to 101006
df_cities["Average Household Income"] = df_cities["Average Household Income"].apply(lambda x: int(x.replace("$", "").replace(",", "")))


In [261]:
df_cities.head()

,City,Average Household Income,location,latitude,longitude
0,Los Angeles,101006,"(Los Angeles, Los Angeles County, CAL Fire Con...",34.053691,-118.242766
1,San Diego,113681,"(San Diego, San Diego County, CAL Fire Souther...",32.717420,-117.162773
2,San Jose,150601,"(San Jose, Santa Clara County, CAL Fire Northe...",37.336166,-121.890591
3,San Francisco,167663,"(San Francisco, CAL Fire Northern Region, Cali...",37.779026,-122.419906
4,Fresno,73396,"(Fresno, Fresno County, CAL Fire Southern Regi...",36.739442,-119.784831


In [262]:
# Convert df_cities to a torch tensor
cities_tensor = torch.tensor(df_cities[["Average Household Income", "latitude", "longitude"]].values).float()
# Reshape
cities_tensor = cities_tensor.reshape(cities_tensor.shape[0], 1, cities_tensor.shape[1])

# preporcess the data
#cities_tensor_processed = preprocess_input(cities_tensor, target_tensor, [], preprocess_transform='power')
#cities_tensor_processed = cities_tensor
# Use a standard scaler
scaler = StandardScaler()
cities_tensor_processed = torch.tensor(scaler.fit_transform(cities_tensor.reshape(-1, cities_tensor.shape[-1])).reshape(cities_tensor.shape))
# convert to float
cities_tensor_processed = cities_tensor_processed.float()

# Pad last dimension with 0 to 100
#TODO: think, do I need to do this or can I just put the vector in the attention?
cities_tensor_processed = torch.nn.functional.pad(cities_tensor_processed, (0, 100-cities_tensor.shape[2]))

In [263]:
# move to GPU
cities_tensor_processed = cities_tensor_processed.to(device)

In [270]:
# import multihead attention
from torch.nn import MultiheadAttention
n = 512
attention = MultiheadAttention(embed_dim=n, num_heads=1, dropout=0.0, device=device)
W_q = torch.rand(n, n).to(device)
W_k = torch.rand(n, n).to(device)
W_v = torch.rand(n, n).to(device)
W_q.requires_grad = True
W_k.requires_grad = True
W_v.requires_grad = True


In [271]:
train_size = 1000
val_size = 2000
test_size = 2000

assert train_size + val_size + test_size == len(california_tensor_processed)

In [272]:
# Suffle the "Average Household Income" column
cities_tensor_processed_random = cities_tensor_processed[torch.randperm(cities_tensor_processed.shape[0])]

In [273]:
# optimize the attention matrix W_q, W_k, W_v
#TODO: check if we train on test
from torch import optim
optimizer = optim.Adam([W_q, W_k, W_v], lr=0.01)
for i in range(100):
    optimizer.zero_grad()
    input = model.model[2].encoder(california_without_income_tensor_processed)
    input2 = model.model[2].encoder(cities_tensor_processed)
    Q = torch.matmul(input, W_q)
    K = torch.matmul(input2, W_k)
    V = torch.matmul(input2, W_v)
    full_input = input + attention(Q, K, V)[0]
    y_pred = module((full_input, target_tensor), single_eval_pos=train_size, x_already_encoded=True).squeeze()
    loss = torch.nn.functional.cross_entropy(y_pred[:val_size], target_tensor[-len(y_pred):-len(y_pred) + val_size].squeeze().long())
    loss.backward()
    optimizer.step()
    print(loss)
    if i % 10 == 0:
        y_pred = torch.argmax(y_pred, dim=-1).reshape(-1)
        print(accuracy_score(target_tensor[-len(y_pred) + val_size:].cpu(), y_pred[val_size:].cpu()))

src_mask None
tensor(0.4882, device='cuda:0', grad_fn=<NllLossBackward>)
0.7615
src_mask None
tensor(0.4905, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4689, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4556, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4462, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4401, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4395, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4360, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4312, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4311, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4279, device='cuda:0', grad_fn=<NllLossBackward>)
0.804
src_mask None
tensor(0.4248, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4286, device='cuda:0', grad_fn=<NllLossBackward>)
src_mask None
tensor(0.4233, device='c

In [ ]:
#TODO: GPU and correct scoring of baselinees